# IPCV Coursework
## Subtask 1 (and 2): 
### a) Image annotation
We automated the process of annotating a test image with the ground truth labeled as purple boundaries by altering the matplotlib example code at https://matplotlib.org/examples/pylab_examples/ginput_manual_clabel.html

In [ ]:
import time
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib
admin
def imshow(image):
    #OpenCV stores images in BGR so we have to convert to RGB to display it using matplotlib
    imagergb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.imshow(imagergb)
    plt.show()

def drawshow(s):
    print(s)
    plt.title(s, fontsize=11)
    plt.draw()
    
i = int(input("Which image? (0,1,...,15) "))
dart = bool(input("Detect faces(0) or dart(1)?"))

image = cv2.imread('./images/dart'+str(i)+ '.jpg')
img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

plt.imshow(img)
plt.waitforbuttonpress()
plt.close()

amount = int(input("How many objects do you want to annotate?: "))

store = np.zeros((amount,2,2), dtype=int)

for k in range(0,amount): 

    plt.imshow(img)

    drawshow('You will annotate the image, click to continue.')

    plt.waitforbuttonpress()
    
    while True:

        pts = []
        while len(pts) < 2:
            drawshow('For each object, draw the boundary by selecting 2 opposite corners with your right click, then press enter.')
            pts = plt.ginput(2, timeout=-1, show_clicks=True)
            
            if len(pts) < 2:
                drawshow('Too few points, starting over.')
                time.sleep(1)  # Wait a second

        cv2.rectangle(img, (int(pts[0][0]),int(pts[0][1])), (int(pts[1][0]),int(pts[1][1])), (128,0,128), 5)
        
        store[k] = pts

        plt.imshow(img)
        drawshow('Done? Enter to continue, mouse click to redo this object. ')

        if plt.waitforbuttonpress(timeout=-1):
            break

        image = cv2.imread('./images/dart'+str(i)+ '.jpg')
        # gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    print(pts)
plt.close()
print(store)

ground = np.zeros((len(store),4),dtype=int)

for k in range (len(store)):
    s = store[k][1] - store[k][0]
    ground[k] = (store[k][0][0], store[k][0][1], s[0],s[1])
    (x,y,w,h) = ground[k]
    cv2.rectangle(image, (x,y), (x+w,y+h), (128,0,128),3)

### b) Automated TPR and F1-score calculation
Using the co-ordinated of the ground truths previously annotated, we run the Viola-Jones face detector and calculate the true positive rate (TPR) and the F1- score of the classification.

In [ ]:
def Compare (A,B):
    Area = np.zeros((len(A),len(B)))
    
    for ai in range (len(A)):
        for bi in range (len(B)):
            (x0,y0,w0,h0) = A[ai]
            (x1,y1,w1,h1) = B[bi]
            TotalA = w0*h0
            if x0<=x1 and y0<=y1:
                for xs in range(x0,x0+w0):
                    for ys in range (y0, y0+h0):
                        if xs==x1 and ys==y1:
                            w = w0-(x1-x0)
                            h = h0-(y1-y0)
                            if w>w1:
                                w = w1
                            if h>h1:
                                h = h1
                            Area[ai,bi] = w*h/TotalA
                            break
            elif x0>x1 and y0>y1:
                for xs in range (x1, x1+w1):
                    for ys in range(y1,y1+h1):
                        if xs==x0 and ys==y0:
                            w = w1-(x0-x1)
                            h = h1-(y0-y1)
                            if w>w0:
                                w = w0
                            if h>h0:
                                h = h0
                            Area[ai,bi] = w*h/TotalA
                            break
            elif x0<=x1:
                for xs in range(x0, x0+w0):
                    if xs==x1:
                            w = w0 - (x1-x0)
                            h = h1 - (y0-y1)
                            if w>w1:
                                w = w1
                            if h>h0:
                                h = h0
                            Area[ai,bi] = w*h/TotalA
                            break
            elif y0<=y1: 
                for xs in range(x1, x1+w1):
                    if xs == x0:
                            w = w1 - (x0-x1)
                            h = h0 - (y1-y0)
                            if w>w0:
                                w = w0
                            if h>h1:
                                h = h1
                            Area[ai,bi] = w*h/TotalA
                            break
            else: 
                print('missing case/condition')            
    return Area

def Eval (A,B,imgcol,thresh=0.4): #Geometric F1 Score
    judge = np.zeros((len(A),len(B)))
    Area = Compare (A,B)
    Areainv = Compare (B,A)
    for a in range (len(A)):
        for b in range (len(B)):
            p = Area[a,b]
            r = Areainv[b,a]
            if (p+r) ==0:
                judge[a,b] = 0
            elif 2*p*r/(p+r)>thresh:
                judge[a,b]= 1            
    return judge

#Viola-Jones detector
if dart:
    classifier = cv2.CascadeClassifier('./Subtask2/classifier/dartcascade/cascade.xml')
else:
    classifier = cv2.CascadeClassifier('./Subtask1/frontalface.xml')
    

img = cv2.imread('./images/dart' + str(i) + '.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

if dart:
    obj = classifier.detectMultiScale(gray, 1.1, 10, 0, (20,20), (500,500))
else:
    obj = classifier.detectMultiScale(gray, 1.1, 1, 0, (50,50), (500,500))

if obj is ():
    print('No objects found')

# Draw box by iteration
for (x,y,w,h) in obj:
    cv2.rectangle(image, (x,y), (x+w,y+h), (0,255,0), 3)

imshow(image)
saveloc = (str("dart" + str(i) + str("classified.jpg")))
cv2.imwrite(saveloc,image)

# Use the evaluator to determine if a classification is a true positive or a false positive.
judge = Eval(ground, obj, image, thresh=0.5)

TP = 0
FN = 0

for k in range (len(judge)):
    sums = np.sum(judge[k])
    if sums == 0: 
        FN += 1
    elif sums >= 1: 
        TP += 1
    else:
        print ('error: missing condition')
FP = len(obj) - TP

detection = [TP, FP, FN]

def f1score(detection):
    tp = detection[0]
    fp = detection[1]
    fn = detection[2]
    precision = tp/(tp + fp)
    recall = tp/(tp + fn)
    f1score = 2*((precision*recall)/(precision+recall))
    return f1score

def tpr(detection):
    tp = detection[0]
    fn = detection[2]
    recall = tp/(tp + fn)
    return recall

print("True Positive Rate of dart" + str(i) + ": ", tpr(detection))
print("F1-score of dart" + str(i) + ": ", f1score(detection))



## Subtask 2: Viola-Jones
### b) The pre-computed (TP,FP,FN) for darts for each image to calculate all F1-scores

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
     
def f1score(detection):
    tp = detection[0]
    fp = detection[1]
    fn = detection[2]
    precision = tp/(tp + fp)
    recall = tp/(tp + fn)
    f1score = 2*((precision*recall)/(precision+recall))
    return f1score

def tpr(detection):
    tp = detection[0]
    fn = detection[2]
    recall = tp/(tp + fn)
    return recall

def ppv(detection):
    tp = detection[0]
    fp = detection[1]
    precision = tp/(tp + fp)
    return precision
    
def fpr(detection):
    fp = detection[1]
    N = detection[3]
    fall_out = fp/N
    return fall_out

#image = [true positive, false positibe, false negative, number of faces]
d0 = [1,0,0]
d1 = [1,0,0]
d2 = [1,2,0]
d3 = [1,2,0]
d4 = [1,0,0]
d5 = [1,0,0]
d6 = [1,0,0]
d7 = [1,1,0]
d8 = [2,0,0]
d9 = [1,3,0]
d10 = [3,8,0]
d11 = [1,1,0]
d12 = [1,0,0]
d13 = [1,3,0]
d14 = [2,11,0]
d15 = [1,0,0]

darts = [d0,d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,d11,d12,d13,d14,d15]

f1 = []
for i in range(0,16):
    if darts[i][0] != 0:
        print("F1-score for d"+str(i)+": ", f1score(darts[i]))
        f1.append(f1score(darts[i]))
    elif darts[i][0] == 0:
        print("F1-score for d"+str(i)+": ", 0)
print ("Average f1: " + str(sum(f1)/len(darts)))

F1-score for d0:  1.0
F1-score for d1:  1.0
F1-score for d2:  0.5
F1-score for d3:  0.5
F1-score for d4:  1.0
F1-score for d5:  1.0
F1-score for d6:  1.0
F1-score for d7:  0.6666666666666666
F1-score for d8:  1.0
F1-score for d9:  0.4
F1-score for d10:  0.42857142857142855
F1-score for d11:  0.6666666666666666
F1-score for d12:  1.0
F1-score for d13:  0.4
F1-score for d14:  0.2666666666666667
F1-score for d15:  1.0
Average f1: 0.7392857142857143


## Subtask 3: Hough Transform and Viola Jones
### b) The pre-computed (TP,FP,FN) for darts for each image to calculate all F1-scores

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
     
def f1score(detection):
    tp = detection[0]
    fp = detection[1]
    fn = detection[2]
    precision = tp/(tp + fp)
    recall = tp/(tp + fn)
    f1score = 2*((precision*recall)/(precision+recall))
    return f1score

def tpr(detection):
    tp = detection[0]
    fn = detection[2]
    recall = tp/(tp + fn)
    return recall

def ppv(detection):
    tp = detection[0]
    fp = detection[1]
    precision = tp/(tp + fp)
    return precision
    
def fpr(detection):
    fp = detection[1]
    N = detection[3]
    fall_out = fp/N
    return fall_out

#image = [true positive, false positibe, false negative, number of faces]
d0 = [1,0,0]
d1 = [1,0,0]
d2 = [1,0,0]
d3 = [1,0,0]
d4 = [1,0,0]
d5 = [1,0,0]
d6 = [0,0,0]
d7 = [1,0,0]
d8 = [1,0,1]
d9 = [0,1,0]
d10 = [2,0,1]
d11 = [1,0,0]
d12 = [1,0,0]
d13 = [1,1,0]
d14 = [2,0,0]
d15 = [1,0,0]

darts = [d0,d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,d11,d12,d13,d14,d15]

f1 = []
for i in range(0,16):
    if darts[i][0] != 0:
        print("F1-score for d"+str(i)+": ", f1score(darts[i]))
        f1.append(f1score(darts[i]))
    elif darts[i][0] == 0:
        print("F1-score for d"+str(i)+": ", 0)
print ("Average f1: " + str(sum(f1)/len(darts)))

F1-score for d0:  1.0
F1-score for d1:  1.0
F1-score for d2:  1.0
F1-score for d3:  1.0
F1-score for d4:  1.0
F1-score for d5:  1.0
F1-score for d6:  0
F1-score for d7:  1.0
F1-score for d8:  0.6666666666666666
F1-score for d9:  0
F1-score for d10:  0.8
F1-score for d11:  1.0
F1-score for d12:  1.0
F1-score for d13:  0.6666666666666666
F1-score for d14:  1.0
F1-score for d15:  1.0
Average f1: 0.8208333333333333
